In [1]:
#https://stackoverflow.com/a/54594071
from IPython.display import HTML

HTML('''<script>
  function code_toggle() {
    if (code_shown){
      $('div.input').hide('500');
      $('#toggleButton').val('Display code')
    } else {
      $('div.input').show('500');
      $('#toggleButton').val('Hide Code')
    }
    code_shown = !code_shown
  }

  $( document ).ready(function(){
    code_shown=false;
    $('div.input').hide()
  });
</script>
''')


# Lat Epig

*The Jupyter Notebook inteface for the LatEpig tool allows you to query all the inscriptions from EDCS in a reproducible manner:  it saves the search result in a TSV file and plots the results on a map of the Roman Empire without any prior knowledge of programming.* 

This programme allows to extracts the output of a search query from the [Epigraphik-Datenbank  Clauss / Slaby (EDCS)](http://www.manfredclauss.de/) in a reproducible manner and saves it as a TSV file. The output can be also plotted the output to a map of the Roman Empire, along with the system of Roman Provinces, roads, and cities. More on used datasets in the *Data Sources* section.  

## Authors 
* [Brian Ballsun-Stanton, Macquarie University, ![](https://orcid.org/sites/default/files/images/orcid_16x16.png)](https://orcid.org/0000-0003-4932-7912)
* [Petra Heřmánková, Aarhus University, ![](https://orcid.org/sites/default/files/images/orcid_16x16.png)](https://orcid.org/0000-0002-6349-0540)
* [Ray Laurence, Macquarie University, ![](https://orcid.org/sites/default/files/images/orcid_16x16.png)](https://orcid.org/0000-0002-8229-1053)

## Cite this software

Ballsun-Stanton B., Heřmánková P., Laurence R. Lat Epig (version 2.0). GitHub. URL: https://github.com/mqAncientHistory/Lat-Epig/ DOI: 10.5281/zenodo.5211341
    
---

## Data sources

### Inscriptions
The Epigraphik-Datenbank Clauss / Slaby (EDCS) is a digital collection of more than 500,000 Latin inscriptions, created by Prof. Manfred Clauss, Anne Kolb, Wolfgang A. Slaby, Barbara Woitas, and hosted by the Universitat Zurich and Katolische Universitat Eichstat-Ingoldstadt. For more see www.manfredclauss.de

### Map
_1. Roman Empire Boundaries & Provinces_

[Ancient World Mapping Centre, political shading shapefiles](http://awmc.unc.edu/awmc/map_data/shapefiles/cultural_data/political_shading/), following the Barington Atlas of Greek Roman World, AWMC.UNC.EDU, under the Creative Commons Attribution-NonCommercial 4.0 International License.

* Roman Empire 60 BC (provinces or extent)
* Roman Empire in AD 14 (provinces or extent)
* Roman Empire in AD 69 (provinces or extent)
* Roman Empire in AD 117 (DEFAULT, provinces or extent)
* Roman Empire in AD 200 (provinces or extent)

_2. Roman Roads_
* McCormick, M. et al. 2013. "Roman Road Network (version 2008)," DARMC Scholarly Data Series, Data Contribution Series #2013-5. DARMC, Center for Geographic Analysis, Harvard University, Cambridge MA 02138.

* [Ancient World Mapping Centre, road shapefiles](http://awmc.unc.edu/awmc/map_data/shapefiles/ba_roads/), shapefile for roads, following the Barington Atlas of Greek Roman World, under the Creative Commons Attribution-NonCommercial 4.0 International License. Collection of shapefiles also vailable through the UCD Digital Library

_3. Cities_

The shapefile of the cities used in the map is based on Hanson, J. W. (2016). Cities Database (OXREP databases). Version 1.0. Accessed (date): http://oxrep.classics.ox.ac.uk/databases/cities/. DOI: https://doi.org/10.5287/bodleian:eqapevAn8. More info available through Hanson, J. W. (2016b). An Urban Geography of the Roman World, 100 B.C. to A.D. 300. Oxford: Archaeopress.

---

## Feedback

If you're using this tool in your research, <!-- Place this tag where you want the button to render. -->
<a class="github-button" href="https://github.com/mqAncientHistory/Lat-Epig" data-color-scheme="no-preference: light; light: light; dark: dark;" data-icon="octicon-star" data-show-count="true" aria-label="Star mqAncientHistory/Lat-Epig on GitHub">Star</a> us on Github! (This way, we don't need to put tracking pixels into this notebook to get a sense of how many folks are using our tool!) 

If you find a bug or have a feature request, raise an <!-- Place this tag where you want the button to render. -->
<a class="github-button" href="https://github.com/mqAncientHistory/Lat-Epig/issues" data-color-scheme="no-preference: light; light: light; dark: dark;" data-icon="octicon-issue-opened" data-show-count="true" aria-label="Issue mqAncientHistory/Lat-Epig on GitHub">Issue</a>!

---


# How to search for inscriptions



### Text search
For simple **Text search** enter your search string into the *Text 1* field. Use Regular Expressions for more complex text string searches, or you can combine search of two terms using *Text 1* and *Text 2* fields, together with *AND*, *OR*, *NOT* operators.

*Examples:* 
* To find all inscriptions containing the term **"tumulus"**, type tumulus in the *Text 1* field.
* To search for word containing **"via"**, type "via". Warning - you will get over 12,000 outputs so it might be a good idea to use Regular Expressions to limit your search, see the following examples.
* To search for any word **starting with "via"**, type " via" (note the space). You will get terms like *via, viator, viatore* ...
* To search for any word **ending with "via"**, type "via " (note the space). You will get terms like *Fulvia, Flavia, Salvia* ...
* To search **only for word "via"**, type " via " (note the spaces).
* To find **two terms**, you can use *Text 1* + *operator* + *Text 2* fields: i.e. to find phrase "viae OR adluvione maris" type into *Text 1* "via", to *Text 2* "adl" and select the "or" *operator*. 

*More about Regular Expressions and how to use them, see the [Carpentries lesson](https://librarycarpentry.org/lc-data-intro/01-regular-expressions/)*

### Additional parameters
If you are interested in other than *Text search* or want to narrow down your searches further, you can use additional fields to specify your search. Those include:

* **EDCS-ID** 
* **Publication**
* **Place**
* **Dating from** (enter as numeric in the following format YYYY, negative values for years BC, positive values for years AD, e.g. number -300 represents year 300 BC, number 56 represents the 56 AD)
* **Dating to** (same format as Dating from)

You can further limit your search by selecting one or more of the **Provinces**, and **Inscription Genus** (custom categorisation of inscriptions applied by EDCS).

Once you are happy with your selection, press the **Get inscriptions!** button and wait for the result to show in the window below.

### Download the results
When the search is done, a total number of inscriptions found will show in the window below, together with two links to download the data in a **TSV file** and a **JSON file** format. Click on the format of your choice to download the link to your local computer. You can easily open a TSV file with your favourite spreadsheet software. For JSON file we recommend using either R or Python.

Note that the *file name* in both formats contains the date of your search, source of the data and how it was accessed (EDCS via LatEpig) and any search parametres or their combinations you have used (*Term 1, Term 2, Dating from*...) and the number of inscriptions found. This way you will always remember what you have searched for and when you share the file with a colleague or students, they can always replicate your search to see if any new inscriptions were added to EDCS.

#### Metadata for the downloaded files

Each file contains the information from EDCS separated into 22 attributes. The [LatEpig Metadata description](https://github.com/mqAncientHistory/Lat-Epig/LatEpig_Metadata_Description.txt) document in the Github repo describes the contents of individual attributes along with their description and original source.

**Happy explorations!**


In [1]:
import warnings
warnings.filterwarnings('ignore')
from lat_epig.interface import makeScrapeInterface, Parseargs                    # loads the user friendly GUI
makeScrapeInterface()

ModuleNotFoundError: No module named 'lat_epig'

# How to export results as a map 

*Once you have found your inscriptions, you can display them on a map of teh Roman Empire and download that map for presentations, publications etc.*

Use the following settings to adjust the map to display the borders of the Roman Empire in a given period, add/remove cities, and add/remove roads.

The search parametres along with other metadata (source, tool used, date, number of results) will be printed on your map for the sake of reproducibility and transparency of your research output.

@Petra continue once fixed

In [3]:
# #https://discourse.jupyter.org/t/check-if-running-on-binder/635/6
# import socket

# hostname = socket.gethostname()
# #print(hostname)

# if "jupyter-mqancienthistor" not in hostname:
from lat_epig.map_interface import make_map_interface
#make_map_interface()
# else:
#     display(HTML("<h1>PDF Map</h1>PDF Map currently disabled on binder because of memory issues. <br/>You should, if you want a PDF map, <a href='https://github.com/mqAncientHistory/EpigraphyScraperNotebook/blob/master/README.md#instructions-how-to-use-the-tool'>Install lat_epig locally in Docker</a> instead."))

print("Currently disabled due to bugs.")


Button(description='Update Data File List', layout=Layout(width='max-content'), style=ButtonStyle())

Dropdown(description='Data File', layout=Layout(width='max-content'), options=(('2022-04-19-EDCS_via_Lat_Epig-…

Text(value='', description='Map Title:')

Dropdown(description='Basemap', index=7, layout=Layout(width='max-content'), options=(('60 BC provinces', 'rom…

RadioButtons(description='Show Provinces', layout=Layout(width='max-content'), options=(('All Provinces', True…

...

RadioButtons(description='Marker size for inscriptions', index=1, layout=Layout(width='max-content'), options=…

Checkbox(value=False, description='Greyscale for publication. (You should probably use a different basemap).',…

RadioButtons(description='Citation', index=1, layout=Layout(width='max-content'), options=(('I will cite this …

Button(description='Export Desired Map!', layout=Layout(width='max-content'), style=ButtonStyle())

Output(layout=Layout(border='1px solid black'))

# How to display an interactive map

*Once you have found your inscriptions, you can display them on here on an interactive map of the Roman Empire. You can then interact with the results and click on the inscriptions to check their text and other available metadata.*

To display an interactive map, there are three simple steps: 
1. Click on the **Update Data File List** to get the most recent searches.
2. Select the desired search output from the **Data File** dropdown.
3. Click on the **Refresh Interactive Map Output** to create a new interactive map and wait a few seconds. The new interactive map should show in the window below. Depending on the number of search outputs, this may take a minute or few (if you are trying to display more than 1000+ inscriptions).

You can zoom-in or zoom-out to cluster or decluster inscriptions and click on the individual markers to display the avialable data or to go directly to EDCS database.

@Brian, look at the go to EDCS link - can it go directly to that one inscription or to the entire search?

In [4]:
from lat_epig.interactive_map_interface import make_i_map_interface
make_i_map_interface()

Button(description='Update Data File List', layout=Layout(width='max-content'), style=ButtonStyle())

Dropdown(description='Data File', layout=Layout(width='max-content'), options=(('2022-04-19-EDCS_via_Lat_Epig-…

Button(description='Refresh Interactive Map!', layout=Layout(width='max-content'), style=ButtonStyle())

Output(layout=Layout(border='1px solid black'))

In [5]:
HTML('''
<form action="javascript:code_toggle()"><input type="submit" id="toggleButton" value="Display code"></form>
<!-- Place this tag in your head or just before your close body tag. -->
<script async defer src="https://buttons.github.io/buttons.js"></script>
''')